In [ ]:
#Import dependencies
import torch
from PIL import Image
from torch import nn, save,load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Load data
train_data= datasets.MNIST(root="/content/gdrive/MyDrive/assignment3_DS",download=False,train=True,transform=ToTensor())
dataset_32=DataLoader(train_data,32) #creat batchs of 32 images

In [ ]:
dataset_1=DataLoader(train_data,1)

In [ ]:
#Exploring the dataset, size, shape..²
for i,d in enumerate(dataset_1):
  print(len(d[0][0][0][0]))
  if i==0:
    break

In [ ]:
#Model Design
class ImageClassifierNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.model=nn.Sequential(
        nn.Conv2d(1,32,(3,3)),
        nn.ReLU(),
        nn.Conv2d(32,64,(3,3)),
        nn.ReLU(),
        nn.Conv2d(64,64,(3,3)),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(64*(28-6)*(28-6),10)
    )
  def forward(self,x):
    return self.model(x)


In [ ]:
#Instiation of the model, loss, optimization
model=ImageClassifierNN().to('cuda') # For gpu utilisation set .to('cuda) 
optimizer=Adam(model.parameters(), lr=1e-3) #(,learning rate) 
loss_fn=nn.CrossEntropyLoss()

In [ ]:
#Implimenting training function --> loop for epochs, and call the model for each dataset element
def train(model,epochs, dataset):
  for epoch in range(epochs):
     total_loss=0
     for batch in dataset:
       X,y=batch
       X,y=X.to('cuda'), y.to('cuda')
       y_pred=model(X)
       loss=loss_fn(y_pred,y)
       #Apply bachpropagation
       optimizer.zero_grad()
       loss.backward()
       optimizer.step()

       total_loss += loss
     print(f"epoch: {epoch} , loss: {total_loss}")
  with open('model_state.pt','wb') as f:
    save(model.state_dict(),f)

In [ ]:
train(model, epochs=20, dataset=dataset_32)

epoch: 0 , loss: 240.99476623535156
epoch: 1 , loss: 85.49656677246094
epoch: 2 , loss: 53.553955078125
epoch: 3 , loss: 36.89036560058594
epoch: 4 , loss: 26.87471580505371
epoch: 5 , loss: 21.610427856445312
epoch: 6 , loss: 16.82542610168457
epoch: 7 , loss: 12.742138862609863
epoch: 8 , loss: 12.518275260925293
epoch: 9 , loss: 9.313807487487793
epoch: 10 , loss: 9.777141571044922
epoch: 11 , loss: 11.734538078308105
epoch: 12 , loss: 6.337721824645996
epoch: 13 , loss: 7.323587417602539
epoch: 14 , loss: 7.6190571784973145
epoch: 15 , loss: 7.392160415649414
epoch: 16 , loss: 5.914453983306885
epoch: 17 , loss: 5.182888984680176
epoch: 18 , loss: 7.213016033172607
epoch: 19 , loss: 6.488308429718018


In [ ]:
type(train_data)

torchvision.datasets.mnist.MNIST

In [ ]:
(train_data[12][1])

3

In [ ]:
random_image = train_data[0][0]
type(random_image)

torch.Tensor

In [ ]:
random_image = train_data[0][0]  #We must be test the model with new data but well....
random_image=random_image.unsqueeze(0).to('cuda')
random_image_label=train_data[0][1]
print(random_image_label)

In [ ]:
#Make prediction
img_pred= torch.argmax(model(random_image))
print(img_pred)

In [ ]:
with open('/content/gdrive/MyDrive/assignment3_DS/model_state.pt','wb') as f:
    save(model.state_dict(),f)

In [ ]:
#Read model:
with open('/content/gdrive/MyDrive/assignment3_DS/model_state.pt','rb') as f:
  model.load_state_dict(load(f))
#If we want to test with another image
img=Image.open('path_to_image_to_predict_its_class')
img_tensor= ToTensor()(img).unsqueeze(0).to('cuda')#Convert image to tensor

img_pred= torch.argmax(model(img_tensor))
print(img_pred)